In [ ]:
!pip uninstall -y torch torchvision torchaudio diffusers transformers accelerate xformers bitsandbytes peft datasets huggingface_hub safetensors

!pip install diffusers==0.30.0 transformers accelerate safetensors torchvision torch --upgrade
!pip install -q diffusers==0.30.0 transformers accelerate safetensors bitsandbytes datasets xformers peft
!pip -q install huggingface_hub datasets

!pip install git+https://github.com/openai/CLIP.git


# !pip install torch==2.6.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --upgrade
# !pip install diffusers==0.30.0 transformers accelerate safetensors xformers==0.0.27.post2 bitsandbytes==0.43.3
# !pip install huggingface_hub datasets peft


Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: diffusers 0.35.2
Uninstalling diffusers-0.35.2:
  Successfully uninstalled diffusers-0.35.2
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import diffusers
import numpy as np
import random

import torch
from huggingface_hub import snapshot_download
from datasets import load_dataset, Image, Value, Features
import os



#smaller dataset
# repo_dir = snapshot_download("inmortalkaktus/pokemon-pixel-art", repo_type="dataset")
# pq = os.path.join(repo_dir, "data", "train-00000-of-00001.parquet")
# features = Features({"image": Image(), "text": Value("string")})
# ds = load_dataset("parquet", data_files=pq, features=features)["train"]

#larger dataset
repo_dir = snapshot_download("ffurfaro/PixelBytes-PokemonAll", repo_type="dataset")
pq = os.path.join(repo_dir, "data", "train-00000-of-00001.parquet")
ds = load_dataset("parquet", data_files=pq)["train"]
ds = ds.remove_columns([c for c in ds.column_names if c not in ["image", "text"]])

print(ds.column_names, len(ds))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/437 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

['image', 'text'] 533


In [ ]:
def seed_everything(seed=2000):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(2000)

In [ ]:
def load_model():

  MODEL_ID = "OFA-Sys/small-stable-diffusion-v0"
  device = "cuda" if torch.cuda.is_available() else "cpu"

  # load the pipeline (FP16 if GPU supports it)
  pipe = diffusers.StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=torch.float16 if device=="cuda" else torch.float32)
  pipe = pipe.to(device)
  # pipe.enable_xformers_memory_efficient_attention() if hasattr(pipe, "enable_xformers_memory_efficient_attention") else None

  try:
      pipe.enable_xformers_memory_efficient_attention()
  except Exception as e:
      print("⚠️ Skipping xformers:", e)

  print("Loaded model:", MODEL_ID, "on", device)


  return pipe, device

pipe, device = load_model()

model_index.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

text_encoder/model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.bin:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--OFA-Sys--small-stable-diffusion-v0/snapshots/38e10e5e71e8fbf717a47a81e7543cd01c1a8140/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--OFA-Sys--small-stable-diffusion-v0/snapshots/38e10e5e71e8fbf717a47a81e7543cd01c1a8140/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The config attributes {'predict_epsilon': True} were passed to DPMSolverMultistepScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--OFA-Sys--small-stable-diffusion-v0/snapshots/38e10e5e71e8fbf717a47a81e7543cd01c1a8140/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--OFA-Sys--smal

Loaded model: OFA-Sys/small-stable-diffusion-v0 on cuda


In [ ]:
save_dir = "/content/pokemon/train"
os.makedirs(save_dir, exist_ok=True)

for i, sample in enumerate(ds):
    # Save image
    img = sample["image"]      # PIL.Image
    img.save(f"{save_dir}/{i:04d}.png")

    # Save caption text
    caption = sample["text"]
    with open(f"{save_dir}/{i:04d}.txt", "w") as f:
        f.write(caption)

print("Done! Saved", len(ds), "images + captions to:", save_dir)

Done! Saved 533 images + captions to: /content/pokemon/train


In [ ]:
# --- ADD LoRA ---

from peft import LoraConfig, get_peft_model
import torch.nn as nn


def lora(pipe, device, RANK=4):

  RANK = 4

  unet = pipe.unet

  lora_config = LoraConfig(
      r=RANK,
      lora_alpha=16,
      target_modules=["to_q", "to_k", "to_v", "to_out.0", "proj_out"],
      lora_dropout=0.05,
      bias="none"
  )

  unet = get_peft_model(unet, lora_config)
  unet.to(device)
  pipe.unet = unet

  trainable = [p for p in unet.parameters() if p.requires_grad]
  print("✅ LoRA attached. Trainable params:", sum(p.numel() for p in trainable))

  return unet, trainable

unet, trainable = lora(pipe, device)

✅ LoRA attached. Trainable params: 573440


In [ ]:
import matplotlib.pyplot as plt
import torch
import clip

# --------------------------------------
# Storage for logs
# --------------------------------------
train_losses = []
val_losses = []
clip_scores = []

# --------------------------------------
# CLIP model for similarity
# --------------------------------------
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

from torchvision.transforms.functional import to_pil_image

def compute_clip_score(imgs, texts):
    """
    imgs: tensor batch in [-1,1]
    texts: list of captions
    """

    # ---- 1. Truncate long captions for CLIP (max context length 77 tokens) ----
    # Hard truncate to ~200 chars (safe for CLIP)
    texts = [t[:200] for t in texts]

    # ---- 2. Convert tensors → PIL ----
    pil_imgs = []
    for img in imgs:
        img = (img * 0.5 + 0.5).clamp(0,1)
        pil_imgs.append(to_pil_image(img.cpu()))

    # ---- 3. Preprocess for CLIP ----
    imgs_pre = torch.stack([clip_preprocess(p) for p in pil_imgs]).to(device)

    # truncate=True prevents CLIP errors
    text_tokens = clip.tokenize(texts, truncate=True).to(device)

    with torch.no_grad():
        img_feat = clip_model.encode_image(imgs_pre)
        txt_feat = clip_model.encode_text(text_tokens)

    img_feat /= img_feat.norm(dim=-1, keepdim=True)
    txt_feat /= txt_feat.norm(dim=-1, keepdim=True)

    similarity = (img_feat * txt_feat).sum(dim=-1)
    return similarity.mean().item()


# --------------------------------------
# Validation step
# --------------------------------------
def validate(val_loader, vae, text_encoder, tokenizer, noise_scheduler):
    unet.eval()
    total_loss = 0
    count = 0

    with torch.no_grad():
        for imgs, captions in val_loader:
            imgs = imgs.to(device, dtype=torch.float16 if device=="cuda" else torch.float32)

            latents = vae.encode(imgs).latent_dist.sample() * 0.18215

            # Add noise
            noise = torch.randn_like(latents)
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps,
                (latents.shape[0],), device=device
            ).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Text encode
            text_inputs = tokenizer(
                captions, padding="max_length", truncation=True,
                max_length=tokenizer.model_max_length,
                return_tensors="pt"
            ).to(device)

            enc = text_encoder(text_inputs.input_ids)[0]

            # Predict noise
            pred = unet(noisy_latents, timesteps, encoder_hidden_states=enc).sample
            loss = ((pred - noise) ** 2).mean()

            total_loss += loss.item()
            count += 1

    unet.train()
    return total_loss / count


# --------------------------------------
# Plotting function
# --------------------------------------
def plot_losses(train_losses, val_losses, learning_rate, weight_decay):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(8,5))
    plt.plot(epochs, train_losses, label="Train Loss")
    plt.plot(epochs, val_losses, label="Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Train vs Validation Loss (lr={learning_rate}, wd={weight_decay})")
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_clip_scores(clip_scores, learning_rate, weight_decay):
    epochs = range(1, len(clip_scores) + 1)
    plt.figure(figsize=(8,5))
    plt.plot(epochs, clip_scores, label="CLIP Score")
    plt.xlabel("Epoch")
    plt.ylabel("CLIP Score")
    plt.title(f"CLIP Score per Epoch (lr={learning_rate}, wd={weight_decay})")
    plt.grid(True)
    plt.show()


100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 187MiB/s]


In [ ]:
class Muon(torch.optim.Optimizer):
    def __init__(self, params, lr=0.02, momentum=0.95, nesterov=True,
                 weight_decay=0, backend='newtonschulz5'):
        if lr <= 0:
            raise ValueError(f"Invalid learning rate: {lr}")
        if not 0 <= momentum < 1:
            raise ValueError(f"Invalid momentum: {momentum}")
        if weight_decay < 0:
            raise ValueError(f"Invalid weight_decay: {weight_decay}")

        defaults = dict(lr=lr, momentum=momentum, nesterov=nesterov,
                       weight_decay=weight_decay)
        super().__init__(params, defaults)
        self.backend = backend

    def _orthogonalize_newton_schulz(self, G, steps=2):
        a, b, c = (3.4445, -4.7750, 2.0315)
        dtype = G.dtype
        G = G.float()
        # scale by spectral norm-ish to keep eigenvalues reasonable
        s = torch.linalg.norm(G)
        if s > 0:
            G = G / s

        for _ in range(steps):
            if G.shape[0] >= G.shape[1]:
                A = G @ G.T
                P = a*A + b*(A@A) + c*(A@A@A)
                G = P @ G
            else:
                A = G.T @ G
                P = a*A + b*(A@A) + c*(A@A@A)
                G = G @ P

        # unscale
        G = G * s
        return G.to(dtype)

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue

                g = p.grad
                state = self.state[p]

                if len(state) == 0:
                    state['momentum_buffer'] = torch.zeros_like(g)

                buf = state['momentum_buffer']
                buf.mul_(group['momentum']).add_(g)

                if group['nesterov']:
                    g = g.add(buf, alpha=group['momentum'])
                else:
                    g = buf

                if len(p.shape) == 2 and self.backend:
                    g = self._orthogonalize_newton_schulz(g)

                if group['weight_decay'] > 0:
                    g.add_(p, alpha=group['weight_decay'])

                p.add_(g, alpha=-group['lr'])

        return loss

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm.auto import tqdm
import torch
from torch.optim import AdamW
import os

class SimpleDataset(Dataset):
    def __init__(self, folder):
        self.files = sorted([f for f in os.listdir(folder) if f.endswith(".png")])
        self.folder = folder
        self.transform = transforms.Compose([
            transforms.Resize((512,512), interpolation=Image.NEAREST),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])
    def __len__(self):
        return len(self.files)
    def __getitem__(self, idx):
        img_path = os.path.join(self.folder, self.files[idx])
        txt_path = img_path.replace(".png", ".txt")

        img = Image.open(img_path).convert("RGB")
        with open(txt_path, "r") as f:
            caption = f.read().strip()
        return self.transform(img), caption


# ============================================
# LOAD BASE MODEL ONCE (outside the function)
# ============================================
pipe, device = load_model()


def train(learning_rate, weight_decay, num_epochs=2, batch_size=4):
    """
    Train with a fresh LoRA each time, but reuse the base model.
    """
    # Fresh LoRA for this run
    unet, trainable = lora(pipe, device)

    dataset = SimpleDataset(save_dir)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    val_size = max(1, len(dataset) // 10)
    train_subset, val_subset = torch.utils.data.random_split(dataset, [len(dataset)-val_size, val_size])
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    optimizer = Muon(trainable, lr=learning_rate, momentum=0.95, weight_decay=weight_decay)
    print("Train with Muon")

    vae = pipe.vae
    noise_scheduler = pipe.scheduler
    tokenizer = pipe.tokenizer
    text_encoder = pipe.text_encoder
    unet.train()

    # Clear logs for new run
    train_losses.clear()
    val_losses.clear()
    clip_scores.clear()

    for epoch in range(num_epochs):

        epoch_loss = 0.0
        batches = 0

        for imgs, captions in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            imgs = imgs.to(device, dtype=torch.float16 if device=="cuda" else torch.float32)

            with torch.no_grad():
                latents = vae.encode(imgs).latent_dist.sample() * 0.18215

            noise = torch.randn_like(latents)
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps,
                (latents.shape[0],), device=device
            ).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            text_inputs = tokenizer(
                captions,
                padding="max_length",
                max_length=tokenizer.model_max_length,
                truncation=True,
                return_tensors="pt"
            ).to(device)

            with torch.no_grad():
                encoder_hidden_states = text_encoder(text_inputs.input_ids)[0]

            pred = unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states).sample
            loss = ((pred - noise) ** 2).mean()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            batches += 1

        avg_train_loss = epoch_loss / batches
        train_losses.append(avg_train_loss)

        # ----- VALIDATION -----
        val_loss = validate(val_loader, vae, text_encoder, tokenizer, noise_scheduler)
        val_losses.append(val_loss)

        # ----- CLIP SCORE -----
        clip_score = compute_clip_score(*next(iter(val_loader)))
        clip_scores.append(clip_score)

        print(f"Epoch {epoch+1}: train={avg_train_loss:.4f}, val={val_loss:.4f}, clip={clip_score:.4f}")

        # Save to Google Drive with hyperparameters in filename
        unet.save_pretrained(f"/content/drive/MyDrive/muon_lr{learning_rate}_wd{weight_decay}_epoch{epoch+1}")


    print("Training complete.")

    # ---------------------------------------------------------
    #                   PLOT RESULTS
    # ---------------------------------------------------------
    plot_losses(train_losses, val_losses, learning_rate, weight_decay)

    print("\n===== FINAL METRICS =====")
    print(f"Training Loss:    {train_losses[-1]:.4f}")
    print(f"Validation Loss:  {val_losses[-1]:.4f}")
    print(f"CLIP Score:       {clip_scores[-1]:.4f}")

    return {
        'train_loss': train_losses[-1],
        'val_loss': val_losses[-1],
        'clip_score': clip_scores[-1],
    }

In [ ]:
lrs = [0.02, 0.01, 0.005, 0.002, 0.001, 0.0001, 0.0005, 0.00001, 0.000005, 0.000001]
wds = [0.1, 0.01, 0.001, 0.0001]
INDIVIDUAL_RUN = False
EPOCHS = 10

result = {}
#hyperparameter tuning
for lr in lrs:
  for wd in wds:
    print(f"RUNNINNG FOR:")
    print(f"Learning Rate: {lr}")
    print(f"Weight Decay: {wd}")
    result[(lr, wd)] = train(
        learning_rate=lr,
        weight_decay=wd,
        num_epochs=EPOCHS
    )
    print("-"*50)

In [ ]:
best_combo = min(result, key=lambda k: result[k]['val_loss'])
best_combo

(1e-05, 0.001)